In [ ]:
%reset -fs

In [ ]:
import pandas as pd
import numpy as np
import os
from pymongo import MongoClient
from dotenv import load_dotenv
import dask.bag as db
from dask_mongo import read_mongo
from dask.distributed import Client

client = Client()

In [ ]:
load_dotenv()

In [ ]:
mongo_pw = os.environ.get("MONGO_PW")

In [ ]:
db_client = MongoClient("mongodb+srv://natedirenzo:" + mongo_pw + "@coolkidscluster.grcwg.mongodb.net/myFirstDatabase?retryWrites=true&w=majority")

In [ ]:
db_client.list_database_names()

In [ ]:
db = db_client['asteroid-db']

In [ ]:
db.list_collection_names()

In [ ]:
collection = db['asteroid-data']

In [ ]:
cursor = collection.find()

In [ ]:
astro_df = pd.DataFrame(list(cursor))